In [2]:
import requests
import datetime
from typing import ClassVar, cast, Dict
from dataclasses import dataclass, asdict
from decimal import Decimal
from lxml import etree
from requests import HTTPError

In [3]:

# Mapeo: Código Corto -> Código Largo
RAW_BANKS_MAPPING = {
    '138': '40138', '133': '40133', '062': '40062', '638': '90638',
    '706': '90706', '659': '90659', '128': '40128', '127': '40127',
    '166': '37166', '030': '40030', '002': '40002', '154': '40154',
    '006': '37006', '137': '40137', '160': '40160', '152': '40152',
    '019': '37019', '147': '40147', '106': '40106', '159': '40159',
    '009': '37009', '072': '40072', '058': '40058', '060': '40060',
    '001': '2001',  '129': '40129', '145': '40145', '012': '40012',
    '112': '40112', '677': '90677', '683': '90683', '630': '90630',
    '124': '40124', '143': '40143', '631': '90631', '901': '90901',
    '903': '90903', '130': '40130', '140': '40140', '652': '90652',
    '688': '90688', '680': '90680', '723': '90723', '722': '90722',
    '720': '90720', '151': '40151', '616': '90616', '634': '90634',
    '689': '90689', '699': '90699', '685': '90685', '601': '90601',
    '168': '37168', '021': '40021', '155': '40155', '036': '40036',
    '902': '90902', '150': '40150', '136': '40136', '059': '40059',
    '110': '40110', '661': '90661', '653': '90653', '670': '90670',
    '602': '90602', '042': '40042', '158': '40158', '600': '90600',
    '108': '40108', '132': '40132', '135': '37135', '710': '90710',
    '684': '90684', '148': '40148', '620': '90620', '156': '40156',
    '014': '40014', '044': '40044', '157': '40157', '728': '90728',
    '646': '90646', '730': '90730', '656': '90656', '617': '90617',
    '605': '90605', '703': '90703', '113': '40113', '141': '40141',
    '715': '90715', '732': '90732', '734': '90734', '167': '40167',
    '721': '90721', '727': '90727', '738': '90738',
}

# Mapeo: Código Largo -> Nombre Oficial
RAW_BANK_NAMES = {
    '40133': 'Actinver', '40062': 'Afirme', '90721': 'Albo', '90706': 'Arcus Fi',
    '90659': 'Asp Integra Opc', '40128': 'Autofin', '40127': 'Azteca', '37166': 'BaBien',
    '40030': 'Bajio', '40002': 'Banamex', '40154': 'Banco Covalto', '37006': 'Bancomext',
    '40137': 'Bancoppel', '40160': 'Banco S3', '40152': 'Bancrea', '37019': 'Banjercito',
    '40147': 'Bankaool', '40106': 'Bank Of America', '40159': 'Bank Of China', '37009': 'Banobras',
    '40072': 'Banorte', '40058': 'Banregio', '40060': 'Bansi', '2001': 'Banxico',
    '40129': 'Barclays', '40145': 'BBase', '40012': 'BBVA Mexico', '40112': 'Bmonex',
    '90677': 'Caja Pop Mexica', '90683': 'Caja Telefonist', '90715': 'Cashi Cuenta', '90630': 'CB Intercam',
    '40124': 'Citi Mexico', '40143': 'CIBanco', '90631': 'CI Bolsa', '90901': 'Cls',
    '90903': 'CoDi Valida', '40130': 'Compartamos', '40140': 'Consubanco', '90652': 'Credicapital',
    '90688': 'Crediclub', '90680': 'Cristobal Colon', '90723': 'Cuenca', '40151': 'Donde',
    '90616': 'Finamex', '90634': 'Fincomun', '90734': 'Finco Pay', '90738': 'Fintoc',
    '90689': 'Fomped', '90699': 'Fondeadora', '90685': 'Fondo (Fira)', '90601': 'Gbm',
    '40167': 'Hey Banco', '37168': 'Hipotecaria Fed', '40021': 'HSBC', '40155': 'Icbc',
    '40036': 'Inbursa', '90902': 'Indeval', '40150': 'Inmobiliario', '40136': 'Intercam Banco',
    '40059': 'Invex', '40110': 'JP Morgan', '90661': 'KLAR', '90653': 'Kuspit',
    '90670': 'Libertad', '90602': 'Masari', '90722': 'Mercado Pago W', '90720': 'MexPago',
    '40042': 'Mifel', '40158': 'Mizuho Bank', '90600': 'Monexcb', '40108': 'Mufg',
    '40132': 'Multiva Banco', '37135': 'Nafin', '90638': 'NU MEXICO', '90710': 'NVIO',
    '40148': 'Pagatodo', '90732': 'Peibo', '90620': 'Profuturo', '40156': 'Sabadell',
    '40014': 'Santander', '40044': 'Scotiabank', '40157': 'Shinhan', '90728': 'Spin by OXXO',
    '90646': 'STP', '90730': 'Swap', '90703': 'Tesored', '90684': 'Transfer',
    '90727': 'Transfer directo', '40138': 'Uala', '90656': 'Unagra', '90617': 'Valmex',
    '90605': 'Value', '40113': 'Ve Por Mas', '40141': 'Volkswagen',
}

# ==========================================
# REEMPLAZO DE CLABE (validación local)
# ==========================================

def normaliza_bank_code(code: str) -> str:
    """
    Devuelve el código LARGO (ej. '40044', '90638').
    Acepta:
      - código corto: '044'  -> '40044' (via RAW_BANKS_MAPPING)
      - código largo: '40044' (si existe en RAW_BANK_NAMES)
      - nombre oficial: 'Scotiabank' (si coincide con RAW_BANK_NAMES.values())
    """
    code = str(code).strip()

    # 1) Si te pasan el nombre (opcional)
    if code in RAW_BANK_NAMES.values():
        for largo, nombre in RAW_BANK_NAMES.items():
            if nombre == code:
                return largo

    # 2) Código corto -> largo
    if code in RAW_BANKS_MAPPING:
        return RAW_BANKS_MAPPING[code]

    # 3) Ya es largo
    if code in RAW_BANK_NAMES:
        return code

    raise ValueError(f"Banco desconocido: {code}")


def validar_banco(code: str) -> str:
    """
    Normaliza y valida. Regresa el código largo si es válido.
    """
    largo = normaliza_bank_code(code)
    # Aquí ya garantizamos que existe en RAW_BANK_NAMES
    return largo


In [4]:

USER_AGENT = (
    'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_14_6) AppleWebKit/537.36 '
    '(KHTML, like Gecko) Chrome/75.0.3770.142 Safari/537.36'
)

BASE_URL = 'https://www.banxico.org.mx/cep'
BASE_URL_BETA = 'https://www.banxico.org.mx/cep-beta'

class Client:
    base_url: ClassVar[str] = BASE_URL

    def __init__(self):
        self.session = requests.Session()
        self.session.headers['User-Agent'] = USER_AGENT
        self.base_data = dict(
            tipoCriterio='T',
            captcha='c',
            tipoConsulta=1,
        )

    def get(self, endpoint: str, **kwargs) -> bytes:
        return self.request('get', endpoint, {}, **kwargs)

    def post(self, endpoint: str, data: dict, **kwargs) -> bytes:
        data = {**self.base_data, **data}
        return self.request('post', endpoint, data, **kwargs)

    def request(
        self, method: str, endpoint: str, data: dict, **kwargs
    ) -> bytes:
        url = Client.base_url + endpoint
        response = self.session.request(method, url, data=data, **kwargs)
        if not response.ok:
            response.raise_for_status()
        return response.content

def configure(beta=False):
    Client.base_url = BASE_URL_BETA if beta else BASE_URL


# ==========================================
# 2. EXCEPCIONES (exc.py)
# ==========================================

class CepError(Exception):
    """
    Error interno del sitio web
    https://www.banxico.org.mx/cep/
    """

class TransferNotFoundError(CepError):
    """
    No se encontró la transferencia con
    los datos proporcionados
    """

class MaxRequestError(CepError):
    """
    Máximo número de peticiones alcanzadas para
    obtener el CEP de una transferencia
    """

class CepNotAvailableError(CepError):
    """
    La transferencia fue encontrada, pero el CEP no
    está disponible.
    """


# ==========================================
# 3. CUENTA (cuenta.py)
# ==========================================

@dataclass
class Cuenta:
    nombre: str
    tipo_cuenta: str
    banco: str
    numero: str
    rfc: str

    @classmethod
    def from_etree(cls, element: etree._Element):
        # Lógica para manejar inconsistencia en atributos de Banxico
        banco = (
            element.attrib['BancoEmisor']
            if 'BancoEmisor' in element.attrib
            else element.attrib['BancoReceptor']
        )

        return cls(
            nombre=element.attrib['Nombre'],
            tipo_cuenta=element.attrib['TipoCuenta'],
            banco=banco,
            numero=element.attrib['Cuenta'],
            rfc=element.attrib['RFC'],
        )


# ==========================================
# 4. TRANSFERENCIA (transferencia.py)
# ==========================================

MAX_REQUEST_ERROR_MESSAGE = (
    b'Lo sentimos, pero ha excedido el n&uacute;mero m&aacute;ximo '
    b'de consultas en este portal'
)

NO_PAYMENT_ERROR_MESSAGE = (
    'No se encontró ningún pago con la información proporcionada'
)

NO_OPERATION_ERROR_MESSAGE = (
    'El SPEI no ha recibido una orden de pago que cumpla con el '
    'criterio de búsqueda especificado'
)

NO_CEP_ERROR_MESSAGE = (
    'Con la información proporcionada se identificó el siguiente pago'
)


@dataclass
class Transferencia:
    fecha_operacion: datetime.date
    fecha_abono: datetime.datetime
    ordenante: Cuenta
    beneficiario: Cuenta
    monto: int  # In cents
    iva: Decimal
    concepto: str
    clave_rastreo: str
    emisor: str
    receptor: str
    sello: str
    tipo_pago: int
    pago_a_banco: bool = False

    @property
    def monto_pesos(self) -> float:
        return self.monto / 100

    @classmethod
    def validar(
        cls,
        fecha: datetime.date,
        clave_rastreo: str,
        emisor: str,
        receptor: str,
        cuenta: str,
        monto: int,
        pago_a_banco: bool = False,
    ):
        client = cls._validar(
            fecha, clave_rastreo, emisor, receptor, cuenta, monto, pago_a_banco
        )

        try:
            xml = cls._descargar(client, 'XML')
        except HTTPError as exc:
            raise CepError from exc

        if MAX_REQUEST_ERROR_MESSAGE in xml:
            raise MaxRequestError

        resp = etree.fromstring(xml)

        ordenante_element = cast(etree._Element, resp.find('Ordenante'))
        beneficiario_element = cast(etree._Element, resp.find('Beneficiario'))

        ordenante = Cuenta.from_etree(ordenante_element)
        beneficiario = Cuenta.from_etree(beneficiario_element)

        cadena_cda = resp.attrib['cadenaCDA'].split("|")

        # FechaAbono is not explicitly provided in response.
        # It can be extracted from the CDA string.
        # Format usually matches indices 4 and 5 in the original pipe-split string
        fecha_abono = datetime.datetime.strptime(
            cadena_cda[4] + cadena_cda[5], '%d%m%Y%H%M%S'
        )
        tipo_pago = cadena_cda[2]

        fecha_operacion = datetime.date.fromisoformat(
            resp.attrib['FechaOperacion']
        )

        iva = beneficiario_element.attrib['IVA']
        concepto = beneficiario_element.attrib['Concepto']
        sello = resp.attrib['sello']

        transferencia = cls(
            fecha_operacion=fecha_operacion,
            fecha_abono=fecha_abono,
            ordenante=ordenante,
            beneficiario=beneficiario,
            monto=monto,
            iva=Decimal(iva),
            concepto=concepto,
            clave_rastreo=clave_rastreo,
            emisor=emisor,
            receptor=receptor,
            sello=sello,
            tipo_pago=int(tipo_pago),
        )
        setattr(transferencia, '__client', client)
        return transferencia

    def descargar(self, formato: str = 'PDF') -> bytes:
        """formato puede ser PDF, XML o ZIP"""
        client = getattr(self, '__client', None)
        if not client:
            client = self._validar(
                self.fecha_abono.date(),
                self.clave_rastreo,
                self.emisor,
                self.receptor,
                self.beneficiario.numero,
                self.monto,
                self.pago_a_banco,
            )
        return self._descargar(client, formato)

    def to_dict(self) -> dict:
        data = asdict(self)
        data['monto_pesos'] = self.monto_pesos
        return data

    @staticmethod
    def _validar(
        fecha: datetime.date,
        clave_rastreo: str,
        emisor: str,
        receptor: str,
        cuenta: str,
        monto: int,
        pago_a_banco: bool = False,
    ) -> Client:
        # Normaliza/valida bancos contra tu catálogo local
        emisor = normaliza_bank_code(emisor)
        receptor = normaliza_bank_code(receptor)

        
        client = Client()  # Use new client to ensure thread-safeness
        request_body = dict(
            fecha=fecha.strftime('%d-%m-%Y'),
            criterio=clave_rastreo,
            emisor=emisor,
            receptor=receptor,
            cuenta=cuenta,
            monto=monto / 100,  # Convert cents to pesos (FLOAT)
            receptorParticipante=1 if pago_a_banco else 0,
        )
        resp = client.post('/valida.do', request_body)
        decoded_resp = resp.decode('utf-8')
        
        if NO_CEP_ERROR_MESSAGE in decoded_resp:
            raise CepNotAvailableError
        if (
            NO_PAYMENT_ERROR_MESSAGE in decoded_resp
            or NO_OPERATION_ERROR_MESSAGE in decoded_resp
        ):
            raise TransferNotFoundError
        return client

    @staticmethod
    def _descargar(client: Client, formato: str = 'PDF') -> bytes:
        """formato puede ser PDF, XML o ZIP"""
        return client.get(f'/descarga.do?formato={formato}')

In [ ]:
tr = Transferencia.validar(
            fecha=datetime.date(2025, 6, 27),
            clave_rastreo='2025062740044B36L0000382565240',
            emisor='40044',  # Emisor
            receptor='90638', # Receptor
            cuenta='638180010119306799', # Clabe
            monto=100  # Centavos
        )

In [6]:
tr

Transferencia(fecha_operacion=datetime.date(2025, 6, 30), fecha_abono=datetime.datetime(2025, 6, 27, 19, 13, 13), ordenante=Cuenta(nombre='RODRIGUEZ OVILLA RICARDO', tipo_cuenta='40', banco='SCOTIABANK     ', numero='044180256032557730', rfc='ROOR9712052X5'), beneficiario=Cuenta(nombre='ULISES RODRIGUEZ OVILLA', tipo_cuenta='40', banco='NU MEXICO      ', numero='638180010119306799', rfc='ROOU000130R77'), monto=100, iva=Decimal('0.00'), concepto='Transferencia a Ulises', clave_rastreo='2025062740044B36L0000382565240', emisor='40044', receptor='90638', sello='AJmtbkhBKTEioW0qirwymMZsK4eTvP+99oFbR62TGLOCltAMZg3TvVyGs7ElEOeXKxulfJeQURfkJhKGkTrp7lmzAUbZo2i0I60iAxhb3D3SHX4ETuy6xZ9FWGQJ29c4ow3XnQCOvCwfADbNTjjnSgKQfIKEMmSOwnSs5jIxZFJGTHHFLY42EHPnDbxc9FmibX48l9hU5tICkycwCcCXQ5uChHxtqbvmH4lNej3bCwnllhTvwY9MtkhUtgeZWToWEqTuZkIinthvKhpr44PLlaB7EEpQEVHF6qXb3edeg25/iuwpdoVK1e6wPvQolmUs8sne9rYS8MKgUbVXD0ZLRg==', tipo_pago=1, pago_a_banco=False)